# Processing of dust data with individual isolation movement
This notebook processes the several data streams needed to run calibration of dust shedding hyperparameters on Spring 2022 data.

In [1]:
using CSV,DataFrames,Plots,Measures,Dates
gr();

ENV["COLUMNS"]=10000;
ENV["LINES"] = 500;

## Inputs

In [ ]:
# Paths the csv files storing Karen's dust collection data
#  Naming convention should be "MmmDd.csv" to match myparsedustfname routine below
fdust = [""];

# Goldstar test file path
fgdstr = "";

# File paths to Chance's spreadsheets for students moving in and out of isolation and bld test compliance
fiso = "";
fcmpl = "";

# Date ranges for which dust is collected
day0 = Date("2022-01-10");
dayf = Date("2022-01-17");

# Days after dayf a later tested positive individual is considered to have contributed to this interval's shedding
daywin = Day(5);

# Buildings considered for independent dust measurement calibration
blds = [""];

## Ancillary functions

As dust data spreadsheets come to have additional month names and associated dates, myparsedustfname may need to be updated below to account.

In [ ]:
function dtrg(v)
    day0 = Date("3000-01-01"); dayf = Date("0000-01-01");
    for i=1:length(v)
        if !ismissing(v[i])
            v0 =  Date(v[i],"m/d/yyyy")
            day0 = v0 < day0 ? v0 : day0;
            
            dayf = v0 > dayf ? v0 : dayf;
        end
    end
    day0 = day0 == Date("3000-01-01") ? missing : day0;
    dayf = dayf == Date("0000-01-01") ? missing : dayf;
    return day0,dayf
end;

function myparsebeg(v)
    return dtrg(v)[1];
end

function myparseend(v)
    return dtrg(v)[2];
end; 

function myuppercase(v)
    if ismissing(v)
        return v
    end
    return uppercase(v)
end;

function mysplit(s::String)
    w=split(s," ")[1]|>Date;
    return w
end;

function myparsedustfname(s::String)
    dt = s[4:5];
    if s[1:3]=="Jan"
        mth = "01";
    elseif s[1:3]=="Feb"
        mth = "02";
    elseif s[1:3]=="Mar"
        mth = "03";
    end
    
    return Date("2022-"*mth*"-"*dt)
end

"""
Used to extract residential buildings from dorm entries of goldstar
"""
function ressplit(s::Union{String,Missing})
    if ismissing(s)
        return s
    end
    val = split(s," - ");
    return convert(String,val[2])
    end;
    
function btwn(x::Date,d1::Date,d2::Date)
    return (x>=d1)&&(x<=d2)
end;

"""
Used to map an iso start date into the range of Chance's dust compliance
"""
function isocmplymap(drm::String,dt::Date,df::DataFrame)
    gdf = groupby(df,"dorm");
    dftemp = gdf[(dorm=drm,)] |> DataFrame;
    sort!(dftemp,"end_date"); dts = dftemp[:,"end_date"];
    n = nrow(dftemp);
    
    dts = [dts[1]-Day(7);dts[:]];
    n+=1;
    
    @inbounds for i=1:n-1
        if (dt>dts[i])&&(dt<=dts[i+1])
            return dts[i+1]
        end
    end
    
    return missing
end
function isocmplymap(drm::Vector{String},dt::Vector{Date},df::DataFrame)
    n = length(drm);
    return [isocmplymap(drm[i],dt[i],df) for i=1:n]
end;

## Process dust sheets

In [ ]:
# Build master dataframe for Karen's dust sheets
dfdustorg = similar(CSV.read(fdust[1],DataFrame),0); dfdustorg[!,"fname"] = [] |> (x->convert(Vector{String},x))
for fname in fdust
    dftemp = CSV.read(fname,DataFrame); dftemp["fname"]=fill(fname,nrow(dftemp));
    dfdustorg = vcat(dfdustorg,dftemp,cols=:union);
end
dfdustorg[!,:Building]=myuppercase.(dfdustorg[!,:Building]);
dfdust = dropmissing(dfdustorg,["Building","Result"]); gdf = groupby(dfdust,["Building","fname"]);
dfdust = combine(gdf,"Start Date"=>myparsebeg=>"Start Date",
                 "Result"=>sum=>"Result (cp/mg)",
                 "End Date"=>myparseend=>"End Date",
                 "fname"=>(x->myparsedustfname.(x))=>"File Date");
select!(dfdust,["Building","Start Date","End Date","File Date","Result (cp/mg)"]); sort!(dfdust,["Building","File Date"]);
rename!(dfdust,["Start Date"=>"Dust Clct Start","End Date"=>"Dust Clct End"]);
dfdust = unique(dfdust);

In [ ]:
println("Building dust signal:")
dfdust

## Process Goldstar and Iso/Bld compliance

In [ ]:
dfmst = CSV.read(fgdstr,DataFrame); 
select!(dfmst,Not("name_n")); dropmissing!(dfmst,["emplid","test_dt","result_dt","campus","general_affil"]);
dfmst[!,"test_dt"] = mysplit.(dfmst[!,"test_dt"]);
dfmst[!,"result_dt"] = mysplit.(dfmst[!,"result_dt"]);
flag = (  (dfmst[!,"test_dt"].>=Date("2022-01-10")).*(dfmst[!,"result_dt"].>=Date("2022-01-10"))
           .*(dfmst[!,"campus"].=="COL").*(dfmst[!,"general_affil"].=="STUDENT")  );
dfmst=dfmst[flag,:]; 
select!(dfmst,["emplid","test_dt","result_dt","result","addr_typ","addr_1"]);

In [ ]:
first(dfmst,7)

#### Filter to just on campus population and positives

In [ ]:
gdf = groupby(dfmst,["addr_typ","result"]);
dfoncmp = gdf[(addr_typ="DORM",result="DETECTED")]|>DataFrame;
dfoncmp[!,"addr_1"] = ressplit.(dfoncmp[!,"addr_1"]);

#### Print the names of campus buildings

In [ ]:
unique(dfoncmp[!,"addr_1"])

### Extract isolation movement and building compliance from Chance's output

#### Individual traffic for the relevant date ranges

In [ ]:
dfiso = CSV.read(fiso,DataFrame); select!(dfiso,Not(["Column1","order_location","dorm_building"]));
dropmissing!(dfiso,"start_date"); #flag = dfchance[!,"start_date"].>=Date("2022-01-10"); dfchance = dfchance[flag,:];
rename!(dfiso,["start_date"=>"isostart_date","release_date"=>"isorelease_date"])
transform!(dfiso,"isostart_date"=>(x->(x.-Date("2022-01-10")))=>"isostart_day_relJan10");
dfindiv = leftjoin(dfoncmp,dfiso,on=:emplid);

# If isostart missing, assume it's one day after the result date
flag = ismissing.(dfindiv[!,"isostart_date"]); 
dfindiv[flag,"isostart_date"] = dfindiv[flag,"result_dt"].+Day(1); dropmissing!(dfindiv,"isostart_date");

# Filter to the relevant date ranges
flag=(dfindiv[!,"isostart_date"].>=day0).*(dfindiv[!,"isostart_date"].<=dayf+daywin);
println("Test positive individuals in all buildings moving into isolation during from $day0 to $daywin after $dayf");
dfindiv=dfindiv[flag,:];sort!(dfindiv,["addr_1","isostart_date"])

#### Extract building compliance from Chance
Confirm with Chance that we should use an end date over a start date

In [ ]:
# Examine Chance's compliance spreadsheet
dfcmpl = CSV.read(fcmpl,DataFrame); select!(dfcmpl,Not("Column1")); dropmissing!(dfcmpl,"dorm");
transform!(dfcmpl,"start_date"=>(x->x.+Day(7))=>"end_date");
transform!(dfcmpl,["n_compliant","n_to_test"]=>((x,y)->x./y)=>"compliance"); sort!(dfcmpl,["dorm","end_date"]);
select!(dfcmpl,["dorm","end_date","n_to_test","compliance"]);

# Filter to relevant date ranges
flag = (dfcmpl[!,"end_date"].>day0).*(dfcmpl[!,"end_date"].<=dayf);
println("Campus compliance across all buildings from ($day0,$dayf]")
dfcmpl = dfcmpl[flag,:]; sort!(dfcmpl,["dorm","end_date"]);
dfcmpl = leftjoin(dfcmpl,dfdust,on=["dorm"=>"Building","end_date"=>"File Date"])

In [ ]:
println("Campus compliance summary statistics:")
gdf = groupby(dfcmpl,"end_date");
dfcmpl_sum = combine(gdf,"compliance"=>(x->sum(x)/length(x))=>"mean campus compliance")

#### Restrict to the relevant buildings for this analysis

In [ ]:
# Process individuals
gdf = groupby(dfindiv,["addr_1"]); dftemp = similar(dfindiv,0);
for bld in blds
    key = (addr_1=bld,);
    dftemp = vcat(dftemp,(gdf[key] |> DataFrame));
end
dfindiv = deepcopy(dftemp);

# Process compliance
gdf = groupby(dfcmpl,["dorm"]); dftemp = similar(dfcmpl,0);
for bld in blds
    key = (dorm=bld,);
    dftemp = vcat(dftemp,(gdf[key] |> DataFrame));
end
dfcmpl = deepcopy(dftemp);

In [ ]:
println("Test positive individuals in specified buildings moving into isolation from [$day0,$dayf+$daywin]:")
dfindiv

In [ ]:
println("Campus compliance in specified buildings from ($day0,$dayf]:")
dfcmpl

#### Renormalize according to compliance for additional number of infected in the building

In [ ]:
# Join individual information with compliance data
transform!(dfindiv,["addr_1","test_dt"]=>((x,y)->isocmplymap(x,y,dfcmpl))=>"end_date");
dftemp = leftjoin(dfindiv,dfcmpl,on=["addr_1"=>"dorm","end_date"=>"end_date"]);

# Aggregate number of observed infected and extrapolated infected during these time periods
gdf = groupby(dftemp,["addr_1","end_date"]);
dfagg = combine(gdf,nrow=>"obs_inf","compliance"=>(x->x[1])=>"compl","n_to_test"=>(x->x[1])=>"pop");
projinf = (pop,cmp,obsinf)->( (obsinf./(pop.*cmp)).*(1 .-cmp).*pop.+obsinf );
transform!(dfagg,["pop","compl","obs_inf"]=>projinf=>"proj_inf");
dropmissing!(dfagg,["end_date","pop","compl"]);
dfagg[!,"proj_inf"] = dfagg[!,"proj_inf"] |> (x->round.(x)) |> (x->Int64.(x));
println("Aggregate counts of infected in the buildings during specified date ranges:")
sort!(dfagg,["addr_1","end_date"]);
dfagg = leftjoin(dfagg,dfdust,on=["addr_1"=>"Building","end_date"=>"File Date"])
select!(dfagg,["addr_1","pop","end_date","compl","obs_inf","proj_inf","Result (cp/mg)"])
CSV.write("blddust.csv",dfagg);
dfagg

## Export Individual infections for these buildings including projected
Even though we have included a 5 day window to account for people shedding before they were detected at a later week, we still assume that those people are observed from the compliant population, ie disjoint from the projected infections above.

In [ ]:
# Prepare a dictionary with keys giving projected infections based on dfagg table
dictpr = Dict{Tuple{String,Date},Int64}();
for i=1:nrow(dfagg)
    bld = dfagg[i,"addr_1"]; dt = dfagg[i,"end_date"]; 
    exinf = dfagg[i,"proj_inf"]-dfagg[i,"obs_inf"];
    dictpr[(bld,dt)]=exinf;
end;

In [ ]:
# Sick any date before test_dt is t₀
# Enter isolation on a isostart_date tℓ
# end_date only used for estimating additional infected individuals
dftemp = select(dfindiv,["addr_1","test_dt","isostart_date","end_date"]);
gdf = groupby(dftemp,["addr_1","end_date"]);

dfexp = similar(dftemp,0); day∞ = Date("2999-12-31");
for dt in unique(dfcmpl[!,"end_date"])
    for bld in blds
        key = (addr_1=bld,end_date=dt);
        local dftemp = gdf[key] |> DataFrame;
        # Fill with dummy rows for the projected infections
        nnew = dictpr[(bld,dt)];
        for n=1:nnew
            dfram = Dict("addr_1"=>[bld],"test_dt"=>[dt],"isostart_date"=>[day∞],
                         "end_date"=>[day∞]) |> DataFrame;
            dftemp = vcat(dftemp,dfram);
        end
        dfexp = vcat(dfexp,dftemp);
    end
end
for bld in blds
    dfexp = vcat(dfexp,DataFrame(gdf[(addr_1=bld,end_date=missing)]));
end
sort!(dfexp,["addr_1","end_date"]);
select!(dfexp,Not("end_date"));

# Add indicator columns to say which individuals count for which dust measurements
for edt in fdust
    sdt = edt[1:5];
    dt = myparsedustfname(edt);
    
    # When handle projected infections localize those to the week they were matched to
    dfexp[!,sdt] = [(dfexp[i,"test_dt"]>=dt-Day(7))&&(dfexp[i,"test_dt"]<=dt+daywin)&&(
                     (dfexp[i,"isostart_date"]!=day∞)||(dfexp[i,"test_dt"]==dt))
                    for i=1:nrow(dfexp)];
end

println("Individual infection and isolation data built from compliance and line data:")
println("(Note: Projected infections had a Dec 31, 2999 date assigned to isostart and their week assigned to test_dt.)")
dfexp

In [ ]:
CSV.write("indivposbybld.csv",dfexp);